In [1]:
%load_ext autoreload
%autoreload 2
import gym
import os
import numpy as np
import random
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from experiment import rollout, ReplayBuffer, Trajectory, load_model, save_model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
MODEL_NAME = 'model_v0_lunar_lander_v2'
HIDDEN = 64
curr_step = 0
writer = SummaryWriter()

class Behavior(torch.nn.Module):
    def __init__(self, input_shape, num_actions):
        super(Behavior, self).__init__()
        self.classifier = torch.nn.Sequential(
            nn.Linear(input_shape, HIDDEN), 
            nn.ReLU(),
            nn.Linear(HIDDEN, HIDDEN),      
            nn.ReLU(),            
            nn.Linear(HIDDEN, HIDDEN),      
            nn.ReLU(),            
            nn.Linear(HIDDEN, HIDDEN),       
            nn.ReLU(),
            nn.Linear(HIDDEN, num_actions)
        )        

    def forward(self, x):
        return self.classifier(x)

In [3]:
env = gym.make('LunarLander-v2')

In [4]:
loss_object = torch.nn.CrossEntropyLoss().to(device)
model_sample = Behavior(input_shape=env.observation_space.shape[0]+2, num_actions=env.action_space.n).to(device)
optimizer = torch.optim.Adam(model_sample.parameters())

In [5]:
rb = ReplayBuffer(max_size=500, last_few=200)

# Random rollout
trajectories, avg_reward, length = rollout(episodes=500, env=env, render=False)
rb.add(trajectories)


# Keep track of steps used during random rollout!
epoch, model_sample, optimizer, loss, steps = load_model(MODEL_NAME, model_sample, optimizer, device, train=True)
steps += length
save_model(MODEL_NAME, epoch, model_sample, optimizer, loss, steps)
writer.add_scalar('Mean_Reward', avg_reward, steps)


print(f"Average Episode Reward: {avg_reward}")

Existing model found. Loading from epoch 7200, steps 188261 with loss: 1.308440923690796
Average Episode Reward: -179.51300150790735


In [6]:
batch_size = 1024

In [7]:
def train_step(model, inputs, targets):
    optimizer.zero_grad()    
    predictions = model(inputs)
    loss = loss_object(predictions, targets)
    
    loss.backward()
    optimizer.step()
    
    return loss

def action_fn(model, inputs, sample_action=True):
    action_logits = model(inputs)
    action_probs = torch.softmax(action_logits, axis=-1)

    if sample_action:        
        m = torch.distributions.categorical.Categorical(logits=action_logits)             
        action = int(m.sample().squeeze().cpu().numpy())        
    else:
        action = int(np.argmax(action_probs.detach().squeeze().numpy()))
    return action
    

In [11]:
# SAMPLE ACTIONS

loss_sum = 0
loss_count = 0

epochs = 1000000
epoch, model_sample, optimizer, loss, steps = load_model(MODEL_NAME, model_sample, optimizer, device, train=True)
print(steps)


for i in range(epoch, epochs+epoch):
    x, y = rb.sample(batch_size, device)    
    loss = train_step(model_sample, x, y)
    loss_sum += loss
    loss_count += 1
    
    writer.add_scalar('Loss/train', loss, i)
    
    if i % 500 == 0:        
        trajectories, mean_reward, length = rollout(100, env=env, model=model_sample, sample_action=True, replay_buffer=rb, 
                              device=device, action_fn=action_fn)
        rb.add(trajectories)
        
        steps += length
        avg_loss = loss_sum/loss_count
        save_model(MODEL_NAME, i, model_sample, optimizer, avg_loss, steps)        
        print(f"Average Episode Reward: {mean_reward}")        
        writer.add_scalar('Mean_Reward', mean_reward, steps)
        

    if i % 200 == 0:
        avg_loss = loss_sum/loss_count
        print(f'i: {i}, s: {steps}, Loss: {avg_loss}')
        
        save_model(MODEL_NAME, i, model_sample, optimizer, avg_loss, steps)

Existing model found. Loading from epoch 123600, steps 4690634 with loss: 0.927336573600769
4690634
i: 123600, s: 4690634, Loss: 0.8920902013778687
i: 123800, s: 4690634, Loss: 0.9279770255088806
Average Episode Reward: -8.499768230625975
i: 124000, s: 4716383, Loss: 0.9281600713729858
i: 124200, s: 4716383, Loss: 0.9322767853736877
i: 124400, s: 4716383, Loss: 0.9341564774513245
Average Episode Reward: -37.61002065796311
i: 124600, s: 4737118, Loss: 0.9357600212097168
i: 124800, s: 4737118, Loss: 0.9370045065879822
Average Episode Reward: -43.43657153080461
i: 125000, s: 4751972, Loss: 0.9377341866493225
i: 125200, s: 4751972, Loss: 0.9389219880104065
i: 125400, s: 4751972, Loss: 0.9393056631088257
Average Episode Reward: 18.344449546997897
i: 125600, s: 4770916, Loss: 0.9400268197059631
i: 125800, s: 4770916, Loss: 0.9409062266349792
Average Episode Reward: -24.647082901031716
i: 126000, s: 4785169, Loss: 0.9417028427124023
i: 126200, s: 4785169, Loss: 0.9424408078193665
i: 126400, s

KeyboardInterrupt: 

In [ ]:
rb.sample_command()

In [ ]:
cmd = (280.085, 318.91295076177306)
rb.sample_command()
env = gym.make('LunarLander-v2')
e, model, _, l = load_model(name=MODEL_NAME, train=False, model=model_sample, optimizer=optimizer, device=device)

_, mean_reward = rollout(episodes=5, env=env, model=model_sample, sample_action=True, 
                      cmd=cmd, render=True, device=device, action_fn=action_fn)


print(f"Average Episode Reward: {mean_reward}")